C2 의 센터를 찾는다.

In [3]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from tqdm import tqdm

from PIL import Image, ImageOps


In [4]:
# DATA_DIR = "/Volumes/SSD970/"
DATA_DIR = "/root/autodl-tmp/cervical_spine/"
SAGITTAL_LABEL_DIR = os.path.join(DATA_DIR, "train_sagittal_labels_jpeg95/")

In [5]:
LABEL_SCALE=32

In [6]:
def read_png_image(path):
    img = Image.open(path)
    return np.asarray(img)

In [7]:
def get_PatientUID(UID):
    return "1.2.826.0.1.3680043." + str(UID)

In [9]:
def read_sagittal_labels(UID, label_scale=256, image_size=256):
    PatientUID = get_PatientUID(UID)
    label_dir = os.path.join(SAGITTAL_LABEL_DIR, PatientUID)

    image_paths = glob.glob(label_dir + "/**")
    image_paths.sort(key=lambda x: int(x.split("/")[-1].replace(".png","")))

    imgs = np.zeros((len(image_paths), image_size, image_size))
    for i, path in enumerate(image_paths):
       np_img = read_png_image(path)
       imgs[i] = np_img

    return imgs / label_scale

# imgs = read_sagittal_labels(9236, LABEL_SCALE)
# print(imgs.max())
# imgs.shape

In [10]:
def pick_spine(imgs, spine=2):
    spine_imgs = np.copy(imgs)
    spine_imgs[spine_imgs != (spine * 0.125)] = 0
    return spine_imgs

In [11]:
def find_center_slice(imgs):
    height, row, column = imgs.nonzero()
    sorted_indice = row.argsort()
    center_pos = sorted_indice[:100].mean()
    return height[int(center_pos)]



## Final method

In [16]:
def find_spine_center(UID, spine, label_scale=256):
    imgs = read_sagittal_labels(UID, label_scale)
    print(imgs.shape)
    imgs = pick_spine(imgs, spine)
    center = find_center_slice(imgs)
    return center, imgs[center, :, :]

center, img = find_spine_center(10001, 2, label_scale=32)
plt.imshow(img, cmap='bone')
plt.axvline(center)

(0, 256, 256)


/var/folders/3l/2d1zfhx90z18_kvtkwwz9xtw0000gn/T/ipykernel_49751/3496154775.py:4: RuntimeWarning: Mean of empty slice.
  center_pos = sorted_indice[:100].mean()


ValueError: cannot convert float NaN to integer